# An analysis of the State of the Union speeches - Part 4

In [85]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from collections import Counter
import shelve

plt.style.use('seaborn-dark')
plt.rcParams['figure.figsize'] = (10, 6)

Again, load data we need from previous parts without redoing that work.

In [86]:
addresses = pd.read_hdf('results/df2.h5', 'addresses')
addresses.head()

,president,title,date,n_sent,n_words_all,n_words,n_uwords,n_swords,n_chars
0,George Washington,State of the Union Address,1790-01-08,24,1178,538,395,356,6753
1,George Washington,State of the Union Address,1790-12-08,40,1515,683,513,463,8455
2,George Washington,State of the Union Address,1791-10-25,60,2487,1136,731,626,14203
3,George Washington,State of the Union Address,1792-11-06,61,2298,1042,682,580,12764
4,George Washington,State of the Union Address,1793-12-03,56,2132,972,714,652,11696


In [87]:
wmat = pd.read_hdf('results/df3.h5', 'wmat')
wmat.head()

,0,1,2,3,4,5,6,7,8,9,...,217,218,219,220,221,222,223,224,225,226
tree-cut,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
18,0,0,0,0,0,0,0,0,0,0,...,1,0,1,1,1,1,0,0,1,0
"134,818",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
osag,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
government—stil,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0


In [88]:
with shelve.open('results/vars3') as db:
    unique_words = db['unique_words']

## Distance between speeches

### By President

Let's first compute a few useful things we'll need later, like the unique set of all presidents and their initials (for plot labeling).

In [89]:
upresidents = addresses['president'].unique()
upres_initials = [p.split()[0][0] + p.split()[1][0] for p in upresidents]
print(upresidents[0:5])
print(upres_initials[0:5])

['  George Washington' '  John Adams' '  Thomas Jefferson'
 '  James Madison' '  James Monroe']
['GW', 'JA', 'TJ', 'JM', 'JM']


Now, make a dataframe that will contain the word counts aggregated by president:

In [90]:
pres_mat = pd.DataFrame()

#gws = addresses.index[addresses['president'] == upresidents[0]]
#pres_mat[upresidents[0]] = wmat[gws].sum(axis=1)

for p in upresidents:
    s = addresses.index[addresses['president'] == p]
    pres_mat[p] = wmat[s].sum(axis=1)
pres_mat.head()

,George Washington,John Adams,Thomas Jefferson,James Madison,James Monroe,John Quincy Adams,Andrew Jackson,Martin van Buren,John Tyler,James Polk,...,Lyndon B. Johnson,Richard Nixon,Gerald R. Ford,Jimmy Carter,Ronald Reagan,George H.W. Bush,William J. Clinton,George W. Bush,Barack Obama,Donald J. Trump
tree-cut,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
18,0,0,2,0,2,0,1,0,0,0,...,1,1,2,1,0,1,3,2,5,0
"134,818",0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
osag,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
government—stil,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


And `pmn` is a normalized version that turns counts into probability distributions:

In [91]:
pmn = pd.DataFrame()

pres_sums = pres_mat.sum(axis=0)
for p in upresidents:
    pmn[p] = pres_mat[p]/pres_sums[p]

pmn.head()

,George Washington,John Adams,Thomas Jefferson,James Madison,James Monroe,John Quincy Adams,Andrew Jackson,Martin van Buren,John Tyler,James Polk,...,Lyndon B. Johnson,Richard Nixon,Gerald R. Ford,Jimmy Carter,Ronald Reagan,George H.W. Bush,William J. Clinton,George W. Bush,Barack Obama,Donald J. Trump
tree-cut,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0
18,0.0,0.0,0.000362,0.0,0.000256,0.0,0.000082,0.000000,0.0,0.0,...,0.000161,0.000247,0.000632,0.000179,0.0,0.000249,0.000284,0.000201,0.000466,0.0
"134,818",0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000153,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0
osag,0.0,0.0,0.000000,0.0,0.000128,0.0,0.000000,0.000000,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0
government—stil,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000093,0.0


Sanity check: all columns should sum to 1.

In [92]:
pmn.sum()

  George Washington        1.0
  John Adams               1.0
  Thomas Jefferson         1.0
  James Madison            1.0
  James Monroe             1.0
  John Quincy Adams        1.0
  Andrew Jackson           1.0
  Martin van Buren         1.0
  John Tyler               1.0
  James Polk               1.0
  Zachary Taylor           1.0
  Millard Fillmore         1.0
  Franklin Pierce          1.0
  James Buchanan           1.0
  Abraham Lincoln          1.0
  Andrew Johnson           1.0
  Ulysses S. Grant         1.0
  Rutherford B. Hayes      1.0
  Chester A. Arthur        1.0
  Grover Cleveland         1.0
  Benjamin Harrison        1.0
  William McKinley         1.0
  Theodore Roosevelt       1.0
  William H. Taft          1.0
  Woodrow Wilson           1.0
  Warren Harding           1.0
  Calvin Coolidge          1.0
  Herbert Hoover           1.0
  Franklin D. Roosevelt    1.0
  Harry S. Truman          1.0
  Dwight D. Eisenhower     1.0
  John F. Kennedy          1.0
  Lyndon

Make a numpy array version to use with Scikit-Learn:

In [93]:
# YOUR CODE HERE
pmm = pd.DataFrame.as_matrix(pmn)
pmm.shape

(19176, 42)

You can use the JSdiv function below as-is, it already works.

In [94]:
from scipy.stats import entropy

def JSdiv(p, q):
    """Jensen-Shannon divergence.
    
    Compute the J-S divergence between two discrete probability distributions.
    
    Parameters
    ----------
    
    p, q : array
        Both p and q should be one-dimensional arrays that can be interpreted as discrete
        probability distributions (i.e. sum(p) == 1; this condition is not checked).
        
    Returns
    -------
    float
        The J-S divergence, computed using the scipy entropy function (with base 2) for
        the Kullback-Leibler divergence.
    """
    m = (p + q) / 2
    return (entropy(p, m, base=2.0) + entropy(q, m, base=2.0)) / 2

This is a "naive" plot where we use the $L^2$ distance to build the embedding.

In [95]:
from sklearn.manifold import MDS

## initialize the distance matrix
#n = pmm.shape[0]
#dist = np.zeros((n,n))
## compute JS distance for all pairs
#for i in range(n):
#    for j in range(n):
#        dist[i,j] = JSdiv(pmm[i,:], pmm[j,:])

## intiialize
#MDS =  MDS()
## transform to lower dimensional representation with Euclidean distance
#lower_naive = MDS.fit_transform(pmm)


# A simple MDS embedding plot:

This will be more informative if we make a little utility function for the plots, that carries a legend, annotates each point with the president's initials, and colors them all:

In [96]:
def plot_embedding(data, title='MDS Embedding', savepath=None, palette='viridis', 
                   size=7, annotate='initials'):
    """Plot an MDS embedding dataframe for all presidents.
    
    Uses Seaborn's `lmplot` to create an x-y scatterplot of the data, encoding the 
    value of the `President` field into the hue (which can be mapped to any desired
    color palette).
    
    Parameters
    ----------
    data : DataFrame
        A DataFrame that must contain 3 columns labeled 'x', 'y' and 'President'.
        
    title : optional, string
        Title for the plot
        
    savepath : optional, string
        If given, a path to save the figure into using matplotlib's `savefig`.
        
    palette : optional, string
        The name of a valid Seaborn palette for coloring the points.
    
    size : optional, float
        Size of the plot in inches (single number, square plot)
        
    annotate: optional, 'initials', 'name' or False
        If 'initials', annotate each point with the intials of each president,
        if 'name', use their last name, and if False, do not annotate at all.
        
    Returns
    -------
    FacetGrid
        The Seaborn FacetGrid object used to create the plot.
    """
    # YOUR CODE HERE

In [97]:
plot_embedding(embed_peu, 'Naive MDS - euclidean distance', 'fig/mds_naive.png', annotate='name');

NameError: name 'embed_peu' is not defined

Now we use a distance metric defined on probability mass functions, the Jensen-Shannon Metric.

In [ ]:
plot_embedding(edf2, 'MDS - Jensen-Shannon Distance', 'fig/mds_jsdiv.png', annotate='name');

### By Speech
First we normalize the term-document matrix

We make a similar naive plot embedding using the $L^2$ distance.

In [ ]:
plot_embedding(embed_df, 'Naive MDS - euclidean distance, all speeches',
               'fig/mds_naive_all.png', size=10);

Now with the Jensen-Shannon metric

In [ ]:
plot_embedding(embed_df2, 'Naive MDS - Jensen-Shannon distance, all speeches', 
               'fig/mds_jdsiv_all.png', size=10);

Store some final results. For native numpy arrays, we can use the convenient numpy `npz` container format, which behaves in practice similar to the Pandas HDF5 store and the Python Shelves:

In [ ]:
np.savez('results/npa4.npz', pmm=pmm)